# Simulate depletion

This notebook process the CoNDERC data that contains both experimental and simulation.

Running this notebook also performs OpenMC depletion simulations for every experiment.

This can take over an hour on a typical laptop but is needed for production of all the results.

In [ ]:
import copy
import shutil

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from urllib.request import urlopen, Request
from zipfile import ZipFile

import openmc
import pypact as pp  # needs latest version which can be installed with pip install git+https://github.com/fispact/pypact

# allows notebook rendering of plotly plots in the HTML made by jupyter-book
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

from openmc_activator import OpenmcActivator, write_markdown_file, read_experimental_data

This downloads and extracts the CoNDERC data. This contains the FNS experimental data and FISPACT inputs and outputs.

In [ ]:
# download zip file
conderc_url = 'https://nds.iaea.org/conderc/fusion/files/fns.zip'
p = Request(conderc_url, headers={'User-Agent': 'Mozilla/5.0'})
with urlopen(p) as response, open('fns.zip', 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

# unzip
with ZipFile('fns.zip', 'r') as f:
    f.extractall('.')

Read all the experiments from unzipped fns folder.

In [ ]:
here = Path('./fns')
assert(here.exists()), 'fns folder does not seem to exist. Run `download_fns_fusion_decay.py` first to download and unzip FNS benchmark files.'
experiments = {}
files = [q for q in here.glob('*') if q.is_dir()]
for f in files:
    if '_' in f.name: continue
    l = list(f.glob('*fluxes*'))
    experiments[f.name] = []
    for name in l:
        x = name.name.replace('_fluxes', '')
        experiments[f.name].append(x)

Reads the Fispact fluxes file that contains the neutron spectra

In [ ]:
# read flux data
flux_dict = {}
for k,l in experiments.items():
    flux_dict[k] = {}
    for exp in l:
        ff = pp.FluxesFile()
        pp.from_file(ff, here / k / (exp+'_fluxes'))
        assert(len(ff.values) == 709)
        ebins = ff.boundaries
        flux_dict[k][exp] = ff.values

Plot and example irradiation neutron spectra.

This example plots the neutron spectra used to irradiate silver (Ag) in the 2000 experimental campaign for 5 minutes of irradiation.

In [ ]:
# in this case we plot the silver Ag experiment spectrum but you could plot others
plt.stairs(values=flux_dict['Ag']['2000exp_5min'], edges=ebins)
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Energy [eV]')  #TODO check these units
plt.ylabel('Flux [n/cm$^2$/s]')  #TODO check these units
plt.close()

Next we read in the experimental data so that it is in a more accessible form.
The times, data and uncertainties are read in.

In [ ]:
# TODO consider replacing with pypact
def is_days(input_file):
    lines = open(input_file, 'r').readlines()
    lines = [q for q in lines if 'TIME' in lines]
    day_cnt = 0
    for line in lines:
        if 'DAYS' in line: day_cnt += 1
    if day_cnt == len(lines):
        return True
    elif day_cnt == 0:
        return False
    else:
        raise ValueError('Something is not right')

exp_data_dict = {'time': {}, 'data': {}, 'uncert': {}}
for k,l in experiments.items():
    for k_ in exp_data_dict:
        exp_data_dict[k_][k] = {}
    for exp in l:
        exp_path = here / k / (exp+'.exp')
        exp_path = str(exp_path.absolute())
        input_path = here / k / ('TENDL-2017_' + exp + '.i')
        input_path = str(input_path.absolute())
        mins, vals, uncs = read_experimental_data(exp_path)
        if is_days(input_path):
            exp_data_dict['time'][k][exp] = (np.array(mins) * 60 * 24).tolist()
        else:
            exp_data_dict['time'][k][exp] = mins
        exp_data_dict['data'][k][exp] = vals # could convert to micro watts (np.array(vals) * 1e6).tolist()
        exp_data_dict['uncert'][k][exp] = uncs
        assert(len(mins) == len(vals))

Now we get the irradiation setup including the flux and timesteps

In [ ]:
def read_irr_setup(filepath):
    ff = pp.InputData()
    pp.from_file(ff, filepath)
    cleaned_irradschedule = [item for item in ff._irradschedule if item != (0.0, 0.0)]
    flux_mag_list = [val[1] for val in cleaned_irradschedule] + [0.0] * len(ff._coolingschedule)
    days_list = np.cumsum([val[0] for val in cleaned_irradschedule] + ff._coolingschedule)/ (24*60*60)
    return days_list.tolist(), flux_mag_list

def read_mat_setup(filepath):
    ff = pp.InputData()
    pp.from_file(ff, filepath)
    return ff._inventorymass.entries

def read_density(filepath):
    ff = pp.InputData()
    pp.from_file(ff, filepath)
    return ff._density

setup_dict = {'days': {}, 'flux_mag': {}, 'mass': {}, 'density': {}}
for k,l in experiments.items():
    for k_ in setup_dict:
        setup_dict[k_][k] = {}
    for exp in l:
        input_path = here / k / ('TENDL-2017_' + exp + '.i')
        input_path = str(input_path.absolute())
        days, flux_mag = read_irr_setup(input_path)
        mass_dict = {k:v/100 for k,v in read_mat_setup(input_path)}
        setup_dict['days'][k][exp] = days
        setup_dict['flux_mag'][k][exp] = flux_mag
        setup_dict['mass'][k][exp] = mass_dict
        setup_dict['density'][k][exp] = read_density(input_path)
        assert(len(days) == len(flux_mag))
        assert(isinstance(mass_dict, dict))
    
            
setup_dict['mg_flux'] = flux_dict
setup_dict['ebins'] = ebins

Now we can carry out depletion simulations in OpenMC

Set the chain file and cross sections to let OpenMC know where to find the data.

The nuclear data used can have an impact on how closely the results match.

To make this a fair comparison we recommend using the same nuclear data as the original Fispact simulations (Tendl 2017) and the chain file provided within the repository.

In [ ]:
# Setting the cross section path to the location used by the CI.
# If you are running this locally you will have to change this path to your local cross section path.
openmc.config['cross_sections'] = Path.home() / 'nuclear_data' / 'cross_sections.xml'

# Setting the chain file to the relative path of the chain file included in the repository.
# Also resolving the chain file to the absolute path which is needed till the next release of OpenMC.
openmc.config['chain_file'] = Path('./fns_spectrum.chain.xml').resolve()

Next we use the experiment descriptions to make OpenMC simulations

The irradiation duration, spectra, flux, material and mass are found from the IAEA Conderc benchmarks and passed to OpenMC functions to perform simulations of the experimental setup.

In [ ]:
openmc_result_dict = {}
all_activation_data = []
element_exp_names = []
for k, l in experiments.items():

    # this loop currently just simulates the all materials in the benchmark suite
    # it can be changed to simulate a single material by commenting the line below.
    # if k != 'Ag': continue
    # or it it can be changed to simulate a two materials by commenting the line below.
    # if k not in ['Ag', 'Al']: continue

    print(f'Running OpenMC for {k} {l}')
    
    if k not in openmc_result_dict:
        openmc_result_dict[k] = {}
    for exp in l:
        if exp in openmc_result_dict[k]:
            continue
        ccfe_flux = flux_dict[k][exp]
        # ebins is ccfs 709 flux bins
        # low to high
        # create new chain file

        # mass in grams
        mass_dict = setup_dict['mass'][k][exp]
        days_list = setup_dict['days'][k][exp]
        # days are cumulative, so we gotta provide diffs
        days_list = np.append(days_list[0], np.diff(days_list))
        flux_mag_list = setup_dict['flux_mag'][k][exp]

        # make openmc material
        mat = openmc.Material()
        for el, md in mass_dict.items():
            el = el.lower().capitalize()
            mat.add_element(el, md, percent_type='wo')
        mat.set_density('g/cm3', setup_dict['density'][k][exp])
        mat.depletable = True
        mat.temperature = 294
        tot_mass = sum(mass_dict.values())
        mat.volume = tot_mass / mat.density

        activation_data = {
            'materials': mat,
            'multigroup_flux': ccfe_flux,
            'energy': ebins,
            'source_rate': flux_mag_list,
            'timesteps': days_list.tolist()
        }
        element_exp_names.append((k,exp))
        all_activation_data.append(activation_data)

In [ ]:
obj = OpenmcActivator(
    activation_data=all_activation_data,
    timestep_units='d',
    chain_file=openmc.config['chain_file'],
)

all_metric_dict = obj.activate(metric_list=['mass', 'decay_heat'])

In [ ]:
for entry, (k,exp) in zip(all_metric_dict, element_exp_names):

    openmc_result_dict[k][exp] = entry

Next we process the Fispact simulations results from the IAEA Conderc benchmarks so that they are ready to plot next to the OpenMC simulation results and the experimental benchmark results.

In [ ]:
def read_fispact_output(filepath):
    lines = open(filepath).readlines()
    # don't read empty lines
    read = False

    lines = [q for q in lines if q.strip()]
    step = 0
    # get header
    # usually the last # line
    for indx, line in enumerate(lines):
        if read:
            spl = line.strip().split()
            now_step = int(spl[0])
            assert(step +1 == now_step)
            step = now_step
            assert(len(spl) == len(col_names)), print(len(spl), len(col_names), '\n', col_names)
            for indx, val in enumerate(spl):
                key = col_names[indx]
                if key == 'step':
                    d[key].append(int(val))
                else:
                    d[key].append(float(val))
            continue
        if line[0] == '#' and lines[indx+1][0] != '#':
            # this line with the column names
            # terrible
            l = line.strip().split()[1:]
            indx = 0
            new_l = []
            while True:
                if indx >= len(l):
                    break
                if l[indx].isalpha():
                    if l[indx+1][-1] == 'm': # metastable
                        if l[indx+1][:-1].isnumeric():
                            new_l.append(l[indx]+l[indx+1])
                            indx += 2
                        else:
                            new_l.append(l[indx])
                            indx += 1
                    else:
                        if l[indx+1].isnumeric(): # metastable
                            new_l.append(l[indx]+l[indx+1])
                            indx += 2
                        else:
                            new_l.append(l[indx])
                            indx += 1
                else:
                    new_l.append(l[indx])
                    indx += 1
            d = {k:[] for k in new_l}
            read = True
            col_names = copy.deepcopy(new_l)
            continue
    # rename total to decay heat for consistence with OpenMC and experimental results dictionaries
    d['decay_heat'] = d.pop('Total')
    d['time'] = (np.array(d['time']) * 365.25 * 60 * 24).tolist() # years to minutes
    return d


fispact_result_dict = {}
for k,l in experiments.items():
    fispact_result_dict[k] = {}
    for exp in l:
        output_path = here / k / f'TENDL-2017_{exp}.nuclides'
        fispact_result_dict[k][exp] = read_fispact_output(output_path.resolve())

We combine the Fispact results (which are per nuclide) so that we have the total values for decay heat.

In [ ]:
fispact_imp_nuclides = {}
for k,l in experiments.items():
    fispact_imp_nuclides[k] = {}
    for exp in l:
        tot = fispact_result_dict[k][exp]['decay_heat']
        indices = [1, len(tot)//2, -1]
        fispact_imp_nuclides[k][exp] = {}
        for i in indices:
            td = {k:v[i] for k,v in fispact_result_dict[k][exp].items() if k not in ['step', 'time', 'uncert', 'decay_heat']}
            td = {k:v for k,v in sorted(td.items(), key=lambda item:item[1], reverse=True)}
            fispact_imp_nuclides[k][exp][i] = td

We define some plotting functions that will be used later

In [ ]:
def plot_with_matplotlib(
    fispact_time,
    fispact_results,
    fispact_uncert,
    openmc_time,
    openmc_results,
    measured_time,
    measured_results,   
    measured_uncert
):
        # plt.errorbar(measured_time, measured, measured_uncert, label='Measured', linestyle='--', marker='x')
        plt.fill_between(
            measured_time,
            measured_results-(3*measured_uncert),
            measured_results+(3*measured_uncert),
            alpha=0.4,
            label='Measured',
            color=(200/255, 200/255, 200/255)
        )
        plt.plot(
            openmc_time,
            openmc_results*1e6,
            label='OpenMC 0.15.3-dev',
            marker='x',
            alpha=0.5,
            color='red'
        )
        plt.errorbar(
            fispact_time,
            fispact_results*1e6,
            fispact_uncert,
            label='FISPACT II',
            marker='o',
            alpha=0.5,
            color='blue'
        )

        plt.yscale('log')
        plt.xlabel('Minutes')
        plt.ylabel(r'Specific heat [$\frac{\mu W}{g}$]')
        plt.legend()
        plt.grid()
        plt.title(f'{k} {exp}')
        plt.savefig(Path('docs') / f'{k}_{exp}.png')
        plt.close()

def plot_with_plotly(
    fispact_time,
    fispact_results,
    fispact_uncert,
    openmc_time,
    openmc_results,
    measured_time,
    measured_results,
    measured_uncert,
    openmc_contributions,
):
    fig = go.Figure()

    # Measured data uncertainty band (3 sigma)
    fig.add_traces([
        go.Scatter(
            x=measured_time,
            y=measured_results - 3 * measured_uncert,
            mode='lines',
            line=dict(width=0),
            showlegend=False,
            hoverinfo='skip'
        ),
        go.Scatter(
            x=measured_time,
            y=measured_results + 3 * measured_uncert,
            mode='lines',
            fill='tonexty',
            fillcolor='rgba(200,200,200,0.4)',
            name='Measured (3σ band)',
            line=dict(width=0),
            hoverinfo='skip'
        )
    ])

    # OpenMC results
    fig.add_trace(go.Scatter(
        x=openmc_time,
        y=openmc_results*1e6,
        mode='lines+markers',
        name='OpenMC 0.15.3-dev',
        marker=dict(symbol='x'),
        line=dict(dash='solid', color='red'),
        opacity=0.7,
    ))

    for key, value in openmc_contributions.items():
        if key.startswith('meta_'):
            continue
        fig.add_trace(go.Scatter(
            x=openmc_time,
            y=np.array(value)*1e6,
            mode='lines',
            name=f'OpenMC {key}',
            line=dict(dash='dash'),
            opacity=0.5,
            visible='legendonly'
        ))

    # FISPACT II results with error bars
    fig.add_trace(go.Scatter(
        x=fispact_time,
        y=fispact_results*1e6,
        mode='markers+lines',
        name='FISPACT II',
        error_y=dict(
            type='data',
            array=fispact_uncert,
            visible=True
        ),
        line=dict(dash='solid', color='blue'),
        marker=dict(symbol='circle'),
        opacity=0.7
    ))

    fig.update_layout(
        yaxis_type="log",
        xaxis_title="Minutes",
        yaxis_title=r"Specific heat [μW/g]",
        legend=dict(title=None),
        title=f"{k} {exp}" if k and exp else "",
        template="plotly_white"
    )
    Path('plotly_files').mkdir(exist_ok=True, parents=True)
    fig.write_html(Path('plotly_files') / f'{k}_{exp}.html')

We now have the OpenMC outputs, Fispact and experimental results in a convenient form  ready for plotting.

The next code block plots the results so that they can be compared.

In [ ]:
for k,l in openmc_result_dict.items():
    for exp in l:
        fispact_time = np.array(fispact_result_dict[k][exp]['time'])
        fispact_uncert = np.array(fispact_result_dict[k][exp]['uncert'])
        fispact_results = np.array(fispact_result_dict[k][exp]['decay_heat'])

        # openmc 
        openmc_time = openmc_result_dict[k][exp]['mass']['meta_time_d']
        decay_indx = 1
        t0 = openmc_time[decay_indx]
        openmc_time = np.array(openmc_time[decay_indx:]) - t0
        openmc_time = openmc_time * (60*24) # days to minutes
        openmc_results = openmc_result_dict[k][exp]['decay_heat']['meta_total']
        mass = openmc_result_dict[k][exp]['mass']['meta_total']
        openmc_results = np.array(openmc_results) / np.array(mass)
        openmc_results = openmc_results[decay_indx:]

        measured = np.array(exp_data_dict['data'][k][exp])
        measured_time = np.array(exp_data_dict['time'][k][exp])
        measured_uncert = np.array(exp_data_dict['uncert'][k][exp])
        # add irradiation time to measured_time

        for index,sorted_dict in fispact_imp_nuclides[k][exp].items():
            print(index, sorted_dict)

        if 'hour' not in exp:
            measured_time = measured_time / (60*24)

        plot_with_plotly(
            fispact_time,
            fispact_results,
            fispact_uncert,
            openmc_time,
            openmc_results,
            measured_time,
            measured,   
            measured_uncert,
            openmc_contributions=openmc_result_dict[k][exp]['decay_heat']
        )
        plot_with_matplotlib(
            fispact_time,
            fispact_results,
            fispact_uncert,
            openmc_time,
            openmc_results,
            measured_time,
            measured,   
            measured_uncert
        )
    # uncomment to write markdown files for each material, needed for local rendering of the jupyter book
    write_markdown_file(
        experiment_names=l,
        material_name=k
    )


In [ ]:
import json
with open('openmc_result_dict.json', 'w') as f:
    json.dump(openmc_result_dict, f, indent=2)
with open('exp_data_dict.json', 'w') as f:
    json.dump(exp_data_dict, f, indent=2)
with open('fispact_result_dict.json', 'w') as f:
    json.dump(fispact_result_dict, f, indent=2)